In [ ]:
import sys
import os
import cv2
import numpy as np
import torch
import pyodbc
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QMainWindow, QPushButton, QFileDialog,
    QTableWidget, QTableWidgetItem, QLabel, QGridLayout, QScrollArea, QMessageBox, QDialog,
    QLineEdit, QHBoxLayout, QHeaderView
)
from PyQt5.QtGui import QPixmap, QFont, QIcon, QImage, QPalette, QBrush
from PyQt5.QtCore import QObject, pyqtSignal, Qt
import pandas as pd

# ==================== CONTAINER PATH CONFIGURATION ====================
BASE_PATH = "/app/Data"

# Image paths
BACKGROUND_IMAGE_MAIN = os.path.join(BASE_PATH, "2.jpeg")
BACKGROUND_IMAGE_STRUCTURE = os.path.join(BASE_PATH, "SFD.png")
LOGIN_IMAGE = os.path.join(BASE_PATH, "human-skeletal.jpg")
THUMBNAIL_IMAGE = os.path.join(BASE_PATH, "thumbnail_1.jpg")

# Model path
MODEL_PATH = os.path.join(BASE_PATH, "exp2/weights/last.pt")

# Data paths
HUMAN_SKELETAL_PATH = os.path.join(BASE_PATH, "Human_Skeletal")
GORILLA_SKELETAL_PATH = os.path.join(BASE_PATH, "Gorilla_Skeletal")
OUR_DATABASE_PATH = os.path.join(BASE_PATH, "Our_DataBase")
ON_DATABASE_PATH = os.path.join(BASE_PATH, "On_DataBase")
DATABASE_PATH = os.path.join(BASE_PATH, "Homam_SK.accdb")

# Model evaluation paths
MODEL_PATHS = {
    'confusion_matrix': os.path.join(BASE_PATH, "model/confusion_matrix"),
    'f1_curve': os.path.join(BASE_PATH, "model/F1_curve"),
    'labels': os.path.join(BASE_PATH, "model/labels"),
    'labels_correlogram': os.path.join(BASE_PATH, "model/labels_correlogram"),
    'p_curve': os.path.join(BASE_PATH, "model/P_curve"),
    'pr_curve': os.path.join(BASE_PATH, "model/PR_curve"),
    'r_curve': os.path.join(BASE_PATH, "model/R_curve"),
    'results': os.path.join(BASE_PATH, "model/results")
}

print(f"Base path: {BASE_PATH}")
print(f"Model path: {MODEL_PATH}")
print(f"Human skeletal path: {HUMAN_SKELETAL_PATH}")

# ==================== ORIGINAL CLASSES (UPDATED WITH NEW PATHS) ====================
class ImageDisplayMixin:
    def display_images_in_grid(self, folder_path, scroll_area_size, image_size, columns=3):
        """Display images from folder in grid layout"""
        # Clear existing layout
        for i in reversed(range(self.layout().count())):
            widget = self.layout().itemAt(i).widget()
            if widget:
                widget.setParent(None)

        # Create scroll area
        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        scroll_area.setFixedSize(*scroll_area_size)
        
        self.layout().addWidget(scroll_area)

        scroll_widget = QWidget()
        scroll_area.setWidget(scroll_widget)
        scroll_widget.setStyleSheet("background-color: black;")

        grid_layout = QGridLayout(scroll_widget)
        grid_layout.setHorizontalSpacing(20)
        grid_layout.setVerticalSpacing(20)
        grid_layout.setColumnStretch(1, 1)

        # Load images from the specified folder path
        if os.path.exists(folder_path):
            images = [img for img in os.listdir(folder_path) 
                     if img.lower().endswith(('.jpg', '.png', '.jpeg'))]
            num_columns = columns
            for index, image_name in enumerate(images):
                row = index // num_columns
                col = index % num_columns
                image_path = os.path.join(folder_path, image_name)
                image_label = QLabel()
                pixmap = QPixmap(image_path)
                if not pixmap.isNull():
                    pixmap = pixmap.scaled(*image_size)
                    image_label.setPixmap(pixmap)
                    grid_layout.addWidget(image_label, row, col)
        else:
            error_label = QLabel(f"Folder not found: {folder_path}")
            error_label.setStyleSheet("color: white; font-size: 16px;")
            grid_layout.addWidget(error_label, 0, 0)

        self.adjustSize()


class Overviewofmodelform(QWidget, ImageDisplayMixin):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Обзор силы модели')
        self.initUI()
        
    def initUI(self):
        # Set the size of the main window
        self.setGeometry(50, 50, 1500, 1200)

        # Load background image
        original_pixmap = QPixmap(BACKGROUND_IMAGE_MAIN)
        pixmap = original_pixmap.scaled(1500, 800)
        brush = QBrush(pixmap)
        palette = self.palette()
        palette.setBrush(QPalette.Background, brush)
        self.setPalette(palette)
        self.setFixedSize(pixmap.size())
        
        layout = QVBoxLayout(self)
        
        # Create buttons
        buttons_config = [
            ('Матрица путаницы', 30, 220, self.open_confusion_matrix_image),
            ('Кривая F1', 260, 150, self.open_F1_curve_image),
            ('Метки', 420, 150, self.open_labels_image),
            ('Коррелограмма \n меток', 585, 200, self.open_labels_correlogram_image),
            ('Кривая P', 800, 150, self.open_P_curve_image),
            ('Кривая PR', 960, 150, self.open_PR_curve_image),
            ('Кривая R', 1120, 150, self.open_R_curve_image),
            ('Результаты', 1280, 150, self.open_results_image)
        ]
        
        for text, x, width, handler in buttons_config:
            button = QPushButton(text, self)
            button.setGeometry(x, 6, width, 60)
            button.setStyleSheet(
                "QPushButton {"
                "   background-color: #B0BBC1; "
                "   color: black; "
                "   border: 2px solid gray; margin-bottom: 0px; margin-top: 0px;"
                "   border-radius: 10px; "
                "}"
                "QPushButton:hover {"
                "   background-color: lightgray; "
                "}"
            )
            button.setFont(QFont("Times New Roman", 14, QFont.Bold))
            button.clicked.connect(handler)
        
        self.setLayout(layout)
        
    def open_confusion_matrix_image(self):
        self.display_images_in_grid(MODEL_PATHS['confusion_matrix'], (1400, 600), (1200, 600))

    def open_F1_curve_image(self):
        self.display_images_in_grid(MODEL_PATHS['f1_curve'], (1400, 600), (1200, 600))
        
    def open_labels_image(self):
        self.display_images_in_grid(MODEL_PATHS['labels'], (1400, 600), (1200, 600))
        
    def open_labels_correlogram_image(self):
        self.display_images_in_grid(MODEL_PATHS['labels_correlogram'], (1400, 600), (1200, 600))
    
    def open_P_curve_image(self):
        self.display_images_in_grid(MODEL_PATHS['p_curve'], (1400, 600), (1200, 600))
    
    def open_PR_curve_image(self):
        self.display_images_in_grid(MODEL_PATHS['pr_curve'], (1400, 600), (1200, 600))
    
    def open_R_curve_image(self):
        self.display_images_in_grid(MODEL_PATHS['r_curve'], (1400, 600), (1200, 600))
        
    def open_results_image(self):
        self.display_images_in_grid(MODEL_PATHS['results'], (1400, 600), (1200, 600))


class StructureOfObjectsForm(QWidget, ImageDisplayMixin):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Структура объектов')
        self.initUI()
        
    def initUI(self):
        # Set the size of the main window
        self.setGeometry(100, 100, 800, 700)

        # Load background image
        original_pixmap = QPixmap(BACKGROUND_IMAGE_STRUCTURE)
        pixmap = original_pixmap.scaled(800, 700)
        brush = QBrush(pixmap)
        palette = self.palette()
        palette.setBrush(QPalette.Background, brush)
        self.setPalette(palette)
        self.setFixedSize(pixmap.size())
        
        layout = QVBoxLayout(self)
        
        self.bayraktar_button = QPushButton('Человеческий скелет', self)
        self.bayraktar_button.setGeometry(130, 6, 290, 60)
        self.bayraktar_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #9F8B79; "
            "   color: black; margin-bottom: 0px; margin-top: 0px; "
            "   border: 2px solid; "
            "   border-radius: 10px; "
            "}"
            "QPushButton:hover {"
            "   background-color: #F4E6D9; "
            "}"
        )
        self.bayraktar_button.setFont(QFont("Times New Roman", 14, QFont.Bold))
        self.bayraktar_button.clicked.connect(self.open_Human_Skeletal_images)

        self.phantom_button = QPushButton('Скелет гориллы', self)
        self.phantom_button.setGeometry(450, 6, 245, 60)
        self.phantom_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #9F8B79; "
            "   color: black; "
            "   border: 2px solid; margin-bottom: 0px; margin-top: 0px;"
            "   border-radius: 10px; "
            "}"
            "QPushButton:hover {"
            "   background-color: #F4E6D9; "
            "}"
        )
        self.phantom_button.setFont(QFont("Times New Roman", 14, QFont.Bold))
        self.phantom_button.clicked.connect(self.open_Gorilla_Skeletal_images)
        
        self.setLayout(layout)
        
    def open_Human_Skeletal_images(self):
        self.display_images_in_grid(HUMAN_SKELETAL_PATH, (800, 500), (600, 600))

    def open_Gorilla_Skeletal_images(self):
        self.display_images_in_grid(GORILLA_SKELETAL_PATH, (800, 500), (600, 600))


class ResultsWindow(QMainWindow):
    def __init__(self, image_path=None, folder_path=None, video_path=None, model_path=MODEL_PATH, image_size=(400, 400)):
        super().__init__()
        self.setWindowTitle('Результаты анализа')
        self.image_size = image_size

        layout = QVBoxLayout()

        # Load model if provided
        if model_path:
            self.model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)
        else:
            self.model = None

        # Analyze single image if provided
        if image_path:
            self.analyze_single_image(image_path, layout)

        # Analyze images in a folder if provided
        if folder_path:
            self.analyze_images_in_folder(folder_path, layout)

        # Analyze video if provided
        if video_path:
            self.analyze_video(video_path, layout)

        central_widget = QWidget()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

    def analyze_single_image(self, image_path, layout):
        # Perform image analysis
        results = self.model(image_path) if self.model else None
        self.display_results(results, layout)

    def analyze_images_in_folder(self, folder_path, layout):
        # Analyze images in the specified folder
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(folder_path, filename)
                results = self.model(image_path) if self.model else None
                self.display_results(results, layout)

    def analyze_video(self, video_path, layout):
        # Perform object detection on the video
        detect_and_save_objects(video_path)
        
    def display_results(self, results, layout):
        if results:
            # Display text results
            text_results = str(results)
            text_label = QLabel(text_results, self)
            layout.addWidget(text_label)

            # Display image with detections
            img_array = np.squeeze(results.render())
            img_array_resized = cv2.resize(img_array, self.image_size)
            height, width, channel = img_array_resized.shape
            bytesPerLine = 3 * width
            qImg = QImage(img_array_resized.data, width, height, bytesPerLine, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qImg)
            img_label = QLabel(self)
            img_label.setPixmap(pixmap)
            layout.addWidget(img_label)


class VideoAnalyzer(QObject):
    video_analysis_finished = pyqtSignal()

    def detect_and_save_objects(self, input_path, new_width=640, new_height=480):
        # Load the YOLOv5 model
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=MODEL_PATH, force_reload=True)

        # Open the video file for reading
        cap = cv2.VideoCapture(input_path)

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Resize the frame
            frame = cv2.resize(frame, (new_width, new_height))

            # Make detections on the frame
            results = model(frame)

            # Extract the detected objects from results
            detected_objects = results.xyxy[0]

            # Save detected objects to the specified path
            for obj in detected_objects:
                x1, y1, x2, y2, confidence, class_id = obj
                object_image = frame[int(y1):int(y2), int(x1):int(x2)]

                # Save the detected object
                object_filename = f"{OUR_DATABASE_PATH}/object_{int(x1)}_{int(y1)}.jpg"
                cv2.imwrite(object_filename, object_image)

            # Display the frame with YOLO detections
            cv2.imshow('Видеоанализ', np.squeeze(results.render()))

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release video capture object
        cap.release()
        cv2.destroyAllWindows()
        self.video_analysis_finished.emit()


class ImageDisplayForm(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('База данных проанализированных изображений')
        self.setGeometry(200, 200, 800, 600)

        layout = QVBoxLayout()

        # Create a scroll area
        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        layout.addWidget(scroll_area)

        # Create a widget for the scroll area
        scroll_widget = QWidget()
        scroll_area.setWidget(scroll_widget)
        scroll_widget.setStyleSheet("background-color: black;")

        # Layout for the images
        grid_layout = QGridLayout(scroll_widget)

        # Load images from folder
        if os.path.exists(ON_DATABASE_PATH):
            images = os.listdir(ON_DATABASE_PATH)
            row, col = 0, 0
            for image_name in images:
                image_path = os.path.join(ON_DATABASE_PATH, image_name)
                image_label = QLabel()
                pixmap = QPixmap(image_path)
                pixmap = pixmap.scaled(400, 400)
                image_label.setPixmap(pixmap)
                grid_layout.addWidget(image_label, row, col)
                col += 1
                if col == 3:
                    col = 0
                    row += 1
        else:
            error_label = QLabel(f"Database path not found: {ON_DATABASE_PATH}")
            grid_layout.addWidget(error_label)

        self.setLayout(layout)


class ImagesChildForm(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Базе данных реального времени')
        self.setGeometry(200, 200, 800, 600)

        layout = QVBoxLayout()

        # Create a scroll area
        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        layout.addWidget(scroll_area)

        # Create a widget for the scroll area
        scroll_widget = QWidget()
        scroll_area.setWidget(scroll_widget)
        scroll_widget.setStyleSheet("background-color: black;")

        # Layout for the images
        grid_layout = QGridLayout(scroll_widget)
        grid_layout.setHorizontalSpacing(20)
        grid_layout.setVerticalSpacing(20)
        grid_layout.setColumnStretch(1, 1)

        # Load images from folder
        if os.path.exists(OUR_DATABASE_PATH):
            images = os.listdir(OUR_DATABASE_PATH)
            num_columns = 3
            for index, image_name in enumerate(images):
                row = index // num_columns
                col = index % num_columns
                image_path = os.path.join(OUR_DATABASE_PATH, image_name)
                image_label = QLabel()
                pixmap = QPixmap(image_path)
                pixmap = pixmap.scaled(300, 300)
                image_label.setPixmap(pixmap)
                grid_layout.addWidget(image_label, row, col)
        else:
            error_label = QLabel(f"Database path not found: {OUR_DATABASE_PATH}")
            grid_layout.addWidget(error_label)

        self.setLayout(layout)


class AccessDatabaseForm(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('База данных обнаружений')
        self.setGeometry(200, 200, 800, 600)
        self.setStyleSheet("background-color: #f0f0f0;")

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        layout = QVBoxLayout()

        # Create a table to display Access database data
        self.table_widget = QTableWidget()
        self.table_widget.setStyleSheet("background-color: #B0BBC1;")
        layout.addWidget(self.table_widget)

        # Load Access database data
        self.load_access_data()

        self.central_widget.setLayout(layout)

    def load_access_data(self):
        # Connect to the Access database
        if os.path.exists(DATABASE_PATH):
            conn_str = f'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={DATABASE_PATH};'
            conn = pyodbc.connect(conn_str)
            cursor = conn.cursor()

            # Execute a query to retrieve data from the Detections table
            cursor.execute("SELECT * FROM Detections")
            rows = cursor.fetchall()

            # Get column names
            columns = [column[0] for column in cursor.description]

            # Display data in the table
            self.table_widget.setRowCount(len(rows))
            self.table_widget.setColumnCount(len(columns))
            self.table_widget.setHorizontalHeaderLabels(columns)
            self.table_widget.horizontalHeader().setStyleSheet("background-color: #d9d9d9;")
            self.table_widget.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)

            for i, row in enumerate(rows):
                for j, value in enumerate(row):
                    cell_value = str(value)
                    item = QTableWidgetItem(cell_value)
                    self.table_widget.setItem(i, j, item)

            # Close the connection
            conn.close()
        else:
            print(f"Database file not found: {DATABASE_PATH}")


class LoginWindow(QDialog):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Вход или регистрация')
        self.setGeometry(200, 200, 1400, 600)

        layout = QGridLayout()
        self.setStyleSheet("background-color: #1d1d1d; color: #f9f9f9;")

        # Login widgets
        login_username_label = QLabel('Имя пользователя:')
        login_password_label = QLabel('Пароль:')
        self.login_username_edit = QLineEdit()
        self.login_password_edit = QLineEdit()
        self.login_password_edit.setEchoMode(QLineEdit.Password)
        login_button = QPushButton('Вход')

        # Apply styling
        labels = [login_username_label, login_password_label]
        for label in labels:
            label.setStyleSheet("font: 25px 'Times New Roman'; color: #f9f9f9; margin-bottom: 5px;")
            
        self.login_username_edit.setStyleSheet("font-size: 25px; padding: 8px; font: bold 25px 'Times New Roman'; border: 2px solid #ccc; border-radius: 10px; background-color: #444; color: #f9f9f9;")
        self.login_password_edit.setStyleSheet("font-size: 25px; padding: 8px; font: bold 25px 'Times New Roman'; border: 2px solid #ccc; border-radius: 10px; background-color: #444; color: #f9f9f9;")

        self.login_username_edit.setFixedWidth(230)
        self.login_password_edit.setFixedWidth(230)
    
        login_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #858585; "
            "   color: white; "
            "   padding: 15px 30px; "
            "   border: none; "
            "   border-radius: 10px; "
            "   font-family: 'Times New Roman'; "
            "   font-weight: bold; "
            "   font-size: 25px; "
            "}"
            "QPushButton:hover {"
            "   background-color: #606060; "
            "}"
        )

        layout.addWidget(login_username_label, 0, 0)
        layout.addWidget(self.login_username_edit, 0, 1)
        layout.addWidget(login_password_label, 1, 0)
        layout.addWidget(self.login_password_edit, 1, 1)
        layout.addWidget(login_button, 2, 1)

        # Registration widgets
        register_username_label = QLabel('Имя пользователя:')
        register_email_label = QLabel('Электронная почта:')
        register_phone_label = QLabel('Номер телефона:')
        register_password_label = QLabel('Пароль:')
        
        self.register_username_edit = QLineEdit()
        self.register_email_edit = QLineEdit()
        self.register_phone_edit = QLineEdit()
        self.register_password_edit = QLineEdit()
        self.register_password_edit.setEchoMode(QLineEdit.Password)
        register_button = QPushButton('Регистрация')

        # Apply styling to registration labels and input fields
        register_labels = [register_username_label, register_email_label, register_phone_label, register_password_label]
        for label in register_labels:
            label.setStyleSheet("font: 25px 'Times New Roman'; color: #f9f9f9; margin-bottom: 5px;")

        register_edit_style = "font: bold 25px 'Times New Roman'; padding: 8px; border: 2px solid #ccc; border-radius: 10px; background-color: #444; color: #f9f9f9;"
        self.register_username_edit.setStyleSheet(register_edit_style)
        self.register_email_edit.setStyleSheet(register_edit_style)
        self.register_phone_edit.setStyleSheet(register_edit_style)
        self.register_password_edit.setStyleSheet(register_edit_style)

        self.register_username_edit.setFixedWidth(300)
        self.register_email_edit.setFixedWidth(300)
        self.register_phone_edit.setFixedWidth(300)
        self.register_password_edit.setFixedWidth(300)
        
        register_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #858585; "
            "   color: white; "
            "   padding: 15px 30px; "
            "   border: none; "
            "   border-radius: 10px; "
            "   font-family: 'Times New Roman'; "
            "   font-weight: bold; "
            "   font-size: 25px; "
            "}"
            "QPushButton:hover {"
            "   background-color: #606060; "
            "}"
        )

        layout.addWidget(register_username_label, 0, 2)
        layout.addWidget(self.register_username_edit, 0, 3)
        layout.addWidget(register_email_label, 1, 2)
        layout.addWidget(self.register_email_edit, 1, 3)
        layout.addWidget(register_phone_label, 2, 2)
        layout.addWidget(self.register_phone_edit, 2, 3)
        layout.addWidget(register_password_label, 3, 2)
        layout.addWidget(self.register_password_edit, 3, 3)
        layout.addWidget(register_button, 4, 3)

        # Add the image
        pixmap = QPixmap(LOGIN_IMAGE)
        pixmap = pixmap.scaled(600, 600)
        image_label = QLabel()
        image_label.setPixmap(pixmap)
        layout.addWidget(image_label, 0, 4, 5, 1)

        self.setLayout(layout)

        # Connect button clicks to functions
        login_button.clicked.connect(self.login)
        register_button.clicked.connect(self.register)

    def login(self):
        username = self.login_username_edit.text()
        password = self.login_password_edit.text()

        if not username or not password:
            QMessageBox.warning(self, 'Вход не выполнен', 'Пожалуйста, введите как имя пользователя, так и пароль.')
            return

        if os.path.exists(DATABASE_PATH):
            try:
                connection_string = f'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={DATABASE_PATH};'
                conn = pyodbc.connect(connection_string)
                cursor = conn.cursor()
                cursor.execute("SELECT Username, Password FROM Users WHERE Username=? AND Password=?", (username, password))
                user = cursor.fetchone()
                if user:
                    self.accept()
                else:
                    QMessageBox.warning(self, 'Вход не выполнен', 'неправильное имя пользователя или пароль')
                cursor.close()
                conn.close()
            except Exception as e:
                QMessageBox.critical(self, 'Ошибка', f'Произошла ошибка: {str(e)}')
        else:
            QMessageBox.critical(self, 'Ошибка', f'База данных не найдена: {DATABASE_PATH}')

    def register(self):
        new_username = self.register_username_edit.text()
        new_password = self.register_password_edit.text()
        new_email = self.register_email_edit.text()
        new_phone = self.register_phone_edit.text()

        if not all([new_username, new_password, new_email, new_phone]):
            QMessageBox.warning(self, 'Регистрация не удалас', 'Пожалуйста заполните все поля.')
            return

        if os.path.exists(DATABASE_PATH):
            try:
                connection_string = f'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={DATABASE_PATH};'
                conn = pyodbc.connect(connection_string)
                cursor = conn.cursor()

                cursor.execute("SELECT Username FROM Users WHERE Username=?", (new_username,))
                existing_user = cursor.fetchone()
                if existing_user:
                    QMessageBox.warning(self, 'Регистрация не удалась', 'Имя пользователя уже занято')
                else:
                    cursor.execute("INSERT INTO Users (Username, Password, Email, Phone_Number) VALUES (?, ?, ?, ?)", (new_username, new_password, new_email, new_phone))
                    conn.commit()
                    QMessageBox.information(self, 'Регистрация прошла успешно', 'Новый пользователь зарегистрирован успешно')
                    self.register_username_edit.clear()
                    self.register_password_edit.clear()
                    self.register_email_edit.clear()
                    self.register_phone_edit.clear()
                cursor.close()
                conn.close()
            except Exception as e:
                QMessageBox.critical(self, 'Ошибка', f'Произошла ошибка: {str(e)}')
        else:
            QMessageBox.critical(self, 'Ошибка', f'База данных не найдена: {DATABASE_PATH}')


class MyWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def initUI(self):
        # Set the size of the main window
        self.setGeometry(100, 100, 1900, 900)

        # Load background image
        original_pixmap = QPixmap(THUMBNAIL_IMAGE)
        pixmap = original_pixmap.scaled(1900, 900)
        brush = QBrush(pixmap)
        palette = self.palette()
        palette.setBrush(QPalette.Background, brush)
        self.setPalette(palette)
        self.setFixedSize(pixmap.size())

        # Create buttons for image and video analysis
        self.image_button = QPushButton('Анализ изображений', self)
        self.image_button.setGeometry(50, 9, 235, 70)
        self.image_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #B0BBC1; "
            "   color: black; "
            "   border: 2px solid gray; margin-bottom: 0px; margin-top: 0px;"
            "   border-radius: 10px; "
            "}"
            "QPushButton:hover {"
            "   background-color: lightgray; "
            "}"
        )
        self.image_button.setFont(QFont("Times New Roman", 14, QFont.Bold))
        self.image_button.clicked.connect(self.open_image_results_window)
        
        self.video_button = QPushButton('Видео анализ', self)
        self.video_button.setGeometry(300, 9, 200, 70)
        self.video_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #B0BBC1; "
            "   color: black; "
            "   border: 2px solid gray; margin-bottom: 0px; margin-top: 0px;"
            "   border-radius: 10px; "
            "}"
            "QPushButton:hover {"
            "   background-color: lightgray; "
            "}"
        )
        self.video_button.setFont(QFont("Times New Roman", 14, QFont.Bold))
        self.video_button.clicked.connect(self.open_video_results_window)
        
        self.database_button = QPushButton('База данных\nобнаружений', self)
        self.database_button.setStyleSheet("QPushButton { text-align: left; }")
        self.database_button.setGeometry(515, 9, 225, 70)
        self.database_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #B0BBC1; "
            "   color: black; "
            "   border: 2px solid gray; margin-bottom: 0px; margin-top: 0px;"
            "   border-radius: 10px; "
            "}"
            "QPushButton:hover {"
            "   background-color: lightgray; "
            "}"
        )
        self.database_button.setFont(QFont("Times New Roman", 14, QFont.Bold))
        self.database_button.clicked.connect(self.open_database_form)
        
        self.child_form_button = QPushButton('База данных\nреального времени', self)
        self.child_form_button.setStyleSheet("QPushButton { text-align: left; }")
        self.child_form_button.setGeometry(750, 9, 240, 70)
        self.child_form_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #B0BBC1; "
            "   color: black; "
            "   border: 2px solid gray; margin-bottom: 0px; margin-top: 0px;"
            "   border-radius: 10px; "
            "}"
            "QPushButton:hover {"
            "   background-color: lightgray; "
            "}"
        )
        self.child_form_button.setFont(QFont("Times New Roman", 14, QFont.Bold))
        self.child_form_button.clicked.connect(self.open_child_form)
        
        self.image_display_button = QPushButton('База данных\nпроанализированных изображений', self)
        self.image_display_button.setStyleSheet("QPushButton { text-align: left; }")
        self.image_display_button.setGeometry(1000, 9, 380, 70)
        self.image_display_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #B0BBC1; "
            "   color: black; "
            "   border: 2px solid gray; margin-bottom: 0px; margin-top: 0px;"
            "   border-radius: 10px; "
            "}"
            "QPushButton:hover {"
            "   background-color: lightgray; "
            "}"
        )
        self.image_display_button.setFont(QFont("Times New Roman", 14, QFont.Bold))
        self.image_display_button.clicked.connect(self.open_image_display_form)
        
        self.structure_button = QPushButton('Структура объектов', self)
        self.structure_button.setGeometry(1390, 9, 230, 70)
        self.structure_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #B0BBC1; "
            "   color: black; "
            "   border: 2px solid gray; margin-bottom: 0px; margin-top: 0px;"
            "   border-radius: 10px; "
            "}"
            "QPushButton:hover {"
            "   background-color: lightgray; "
            "}"
        )
        self.structure_button.setFont(QFont("Times New Roman", 14, QFont.Bold))
        self.structure_button.clicked.connect(self.open_structure_form)
        
        self.model_button = QPushButton('Обзор силы модели', self)
        self.model_button.setGeometry(1630, 9, 250, 70)
        self.model_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #B0BBC1; "
            "   color: black; "
            "   border: 2px solid gray; margin-bottom: 0px; margin-top: 0px;"
            "   border-radius: 10px; "
            "}"
            "QPushButton:hover {"
            "   background-color: lightgray; "
            "}"
        )
        self.model_button.setFont(QFont("Times New Roman", 14, QFont.Bold))
        self.model_button.clicked.connect(self.open_model_form)

        self.setWindowTitle('SkeletonAnalyzerPro - Анализ скелетных структур человека и гориллы')
        self.show()

    def open_model_form(self):
        self.Overview_of_model_form = Overviewofmodelform()
        self.Overview_of_model_form.show()
        
    def open_image_results_window(self):
        options = QFileDialog.Options()
        file_dialog = QFileDialog()
        file_dialog.setFileMode(QFileDialog.ExistingFiles)
        file_dialog.setNameFilter("Images (*.jpg *.png)")
        filenames, _ = file_dialog.getOpenFileNames(self, "Выберите изображение или папку", "", "Images (*.jpg *.png);;All Files (*)", options=options)

        if filenames:
            image_size = (800, 600)
            self.results_window = ResultsWindow(image_path=filenames[0], model_path=MODEL_PATH, image_size=image_size)
            self.results_window.show()

    def open_video_results_window(self):
        options = QFileDialog.Options()
        file_dialog = QFileDialog()
        file_dialog.setFileMode(QFileDialog.ExistingFiles)
        file_dialog.setNameFilter("Videos (*.mp4 *.avi)")
        filenames, _ = file_dialog.getOpenFileNames(self, "Выберите видео", "", "Videos (*.mp4 *.avi)", options=options)

        if filenames:
            self.video_analyzer = VideoAnalyzer()
            self.video_analyzer.video_analysis_finished.connect(self.open_child_form)
            self.video_analyzer.detect_and_save_objects(filenames[0])

    def open_child_form(self):
        self.child_form = ImagesChildForm()
        self.child_form.show()
        
    def open_database_form(self):
        self.database_form = AccessDatabaseForm()
        self.database_form.show()
        
    def open_image_display_form(self):
        self.image_display_form = ImageDisplayForm()
        self.image_display_form.show()
        
    def open_structure_form(self):
        self.structure_of_objects_form = StructureOfObjectsForm()
        self.structure_of_objects_form.show()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    login_window = LoginWindow()
    if login_window.exec_() == QDialog.Accepted:
        main_window = MyWindow()
        main_window.show()
        sys.exit(app.exec_())